In [2]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=451d19c979be0e2a2962455d2407be423b05f130a0463e632bde1c93d526d0e2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [3]:
#### FINAL SENTENCE-BERT BASET RANKER ####
import torch
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def calculate_similarity(query_embedding, database_embedding):
    ''' Cosine Similarity function '''
    if len(query_embedding.shape) == 1:
        query_embedding = query_embedding.unsqueeze(0)
    if len(database_embedding.shape) == 1:
        database_embedding = database_embedding.unsqueeze(0)

    return torch.nn.functional.cosine_similarity(query_embedding, database_embedding, dim=1).numpy()


#### Cleaning and initialising the Database of found items. Also adding the vector embeddings ###
database = [
    {'ItemID': 1, 'Description': "Found a red wallet in the hostel area"},
    {'ItemID': 2, 'Description': "Found black phone near the park"},
    {'ItemID': 3, 'Description': "Found kerys in the hostel area"},
    {'ItemID': 4, 'Description': "One Red & Blue Colour Pendrive (SANDISK) found in AB-03"},
    {'ItemID': 5, 'Description': "Ten Rupees foind in AB-03"},
    {'ItemID': 6, 'Description': "One Blue & Black Colour Plastic Water Bottle (Stryder H2o) found in Kyzeel Hostel"},
    {'ItemID': 7, 'Description': "Found a maruti suzuki car key near Amul Store"},
    {'ItemID': 8, 'Description': "Found a black wallet next to Amul Store. It has some cash and some cards."},
    {'ItemID': 9, 'Description': "(seven) keys were found from Sports Complex Cricket Ground near Mango tree"},
    {'ItemID': 10, 'Description': "(10) Keys were found behind AB-1 Two degree cafe"},
    {'ItemID': 11, 'Description': "Found one Black Colour Steel Water Bottle (IIT LOGO) in AB-03"},
    {'ItemID': 12, 'Description': "Found One Grey Colour Measuring Tape 30m (Freemans) in AB-07"},
    {'ItemID': 13, 'Description': "One Black & Gold Colour Chain in Aibaan hostel"},
    {'ItemID': 14, 'Description': "One Black & Red Colour Earphone (Costar) found in hiqom"},
    {'ItemID': 15, 'Description': "One Silver Colour Steel Water Bottle (From 29 Seater Bus) at Gate 1"},
    {'ItemID': 16, 'Description': "One Grey Colour USB-C Fusion Max-6 (ALOGIC) found in AB-05"},
    {'ItemID': 17, 'Description': "Foun one Blck Colour Cycle Number Lock at gate 1"},
    {'ItemID': 18, 'Description': "One Silver Colour Chain With Krishna Locket in Aibaan hostel."}
]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

for item in database:
    description = item['Description']
    words = word_tokenize(description)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    filtered_description = ' '.join(filtered_words)
    embedding = model.encode(filtered_description, convert_to_tensor=True)
    item['Embedding'] = embedding

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [4]:
### Taking User Query and Pre-processing it ###
user_query = "Chain with photograph of lord Krishna"
query_words = word_tokenize(user_query)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
filtered_words = [lemmatizer.lemmatize(word.lower()) for word in query_words if word.lower() not in stop_words]
filtered_query = ' '.join(filtered_words)

### Making vector embedding of the user query ###
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
user_query_embedding = model.encode(filtered_query, convert_to_tensor=True)

#### Creating a Ranking ###
minThresh = 0.40
result = []
for item in database:
    description_embedding = item['Embedding']
    similarity_score = calculate_similarity(user_query_embedding, description_embedding)
    if (similarity_score > minThresh):
      result.append([item['ItemID'], item['Description'], float(similarity_score)])
result.sort(key=lambda x: x[2], reverse=True)
if (len(result) != 0):
    for item_id, description, similarity_score in result :
        print(f"Item ID: {item_id}, Descriptioon: {description}, Cosine Similarity Score: {similarity_score}")
else:
    print("No matching items found. Please try again later.")


Item ID: 18, Descriptioon: One Silver Colour Chain With Krishna Locket in Aibaan hostel., Cosine Similarity Score: 0.42934244871139526
